# Hierarchical Probabilistic Count Matrix Factorization

In [1]:
%matplotlib inline

import sys
sys.path.insert(0, '../')
import utils, cavi_new, svi_new, cavi_hpcmf, svi_hpcmf
import plot_utils

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split

from scipy.stats import gamma

import operator

plt.style.use('ggplot')

In [2]:
# Experiment parameters
N = 1000 # number of observations
P = 10 # observation space dimensionality
K = 2 # latent space dimensionality
C = 2 # number of clusters

# Generate data set
z_p = 0.5
eps = 5.
Y, D, X, R, V, U, clusters = utils.generate_data(N, P, K, C=C, zero_prob=z_p, 
                                                 eps=eps, return_all=True)

Y_train, Y_test, U_train, U_test, c_train, c_test = train_test_split(Y, U.T, clusters, test_size=0.2, random_state=42)

In [3]:
T = 30.
S = 1.

# Run PCA
pca_U = PCA(n_components=2).fit_transform(Y_train)

# # Run SVI and get estimates (hpCMF)
# alpha = np.ones((2, K))
# alpha[0, :] = 3.
# alpha[1, :] = 0.5
# beta = np.ones((2, P, K))
# pi = np.ones((P,)) * 0.5
# print('SHPCMF:')
# inf = svi_hpcmf.StochasticVI(Y_train, alpha, beta, pi)
# shpcmf_ll = inf.run_svi(n_iterations=4000, minibatch_size=100, return_ll=True, sampling_rate=S, max_time=T)
# shpcmf_U = inf.a[0] / inf.a[1] # VI estimate is the mean of the variational approximation

# print('\n')

# # Run CAVI and get estimates (hpCMF)
# alpha = np.ones((2, K))
# alpha[0, :] = 3.
# alpha[1, :] = 0.5
# beta = np.ones((2, P, K))
# pi = np.ones((P,)) * 0.5
# print('HPCMF:')
# inf = cavi_hpcmf.CoordinateAscentVI(Y_train, alpha, beta, pi)
# hpcmf_ll = inf.run_cavi(n_iterations=4000, return_ll=True, sampling_rate=S, max_time=T)
# hpcmf_U = inf.a[0] / inf.a[1] # VI estimate is the mean of the variational approximation

# print('\n')

# # Run CAVI and get estimates (pCMF)
# alpha = np.ones((2, K))
# alpha[0, :] = 3.
# alpha[1, :] = 0.5
# beta = np.ones((2, P, K))
# pi = np.ones((P,)) * 0.5
# print('PCMF:')
# inf = cavi_new.CoordinateAscentVI(Y_train, alpha, beta, pi)
# pcmf_ll = inf.run_cavi(n_iterations=4000, empirical_bayes=True, return_ll=True, sampling_rate=S, max_time=T)
# pcmf_U = inf.a[0] / inf.a[1] # VI estimate is the mean of the variational approximation

# print('\n')

# Run SVI and get estimates (pCMF)
alpha = np.ones((2, K))
alpha[0, :] = 3.
alpha[1, :] = 0.5
beta = np.ones((2, P, K))
pi = np.ones((P,)) * 0.5
print('SPCMF:')
inf = svi_new.StochasticVI(Y_train, alpha, beta, pi)
spcmf_ll = inf.run_svi(n_iterations=4000, minibatch_size=100, empirical_bayes=True, return_ll=True, sampling_rate=S, max_time=T)
spcmf_U = inf.a[0] / inf.a[1] # VI estimate is the mean of the variational approximation

SPCMF:


NameError: name 'exit' is not defined

In [6]:
inf.alpha[0, :, :]

array([[-350835.23856866, -337956.56030722],
       [-350835.23856866, -337956.56030722],
       [-350835.23856866, -337956.56030722],
       ...,
       [-350835.23856866, -337956.56030722],
       [-350835.23856866, -337956.56030722],
       [-350835.23856866, -337956.56030722]])

In [ ]:
from utils import psi_inverse

In [ ]:
psi_inverse(2., inf.alpha[0, 0, 0])

In [ ]:
plt.plot(shpcmf_ll[1], label='SHPCMF')
plt.plot(hpcmf_ll[1], label='HPCMF')
plt.plot(pcmf_ll[1], label='PCMF')
plt.plot(spcmf_ll[1], label='SPCMF')
plt.ylabel('Average log-likelihood')
plt.xlabel('Seconds')
plt.title('Data set with N={} and P={}'.format(N, P))
plt.legend()
plt.show()

In [ ]:
U_list = [U_train, pcmf_U, spcmf_U, hpcmf_U, shpcmf_U, pca_U]
title_list = ['True $\mathbf{U}$', 'PCMF', 'SPCMF', 'HPCMF', 'SHPCMF', 'PCA']

n_results = len(U_list)

assert len(U_list) == len(title_list)

fig = plt.figure(figsize=(16, 10))

s = 30
alpha = 0.7
labels=None
for i in range(len(U_list)):
    ax = plt.subplot(2, n_results/2, i+1)
    handlers = []
    for c in range(C):
        h = ax.scatter(U_list[i][c_train==c, 0], U_list[i][c_train==c, 1], s=s, alpha=alpha)
        handlers.append(h)
    if labels is not None:
        ax.legend(handlers, labels, scatterpoints=1)
    plt.title(title_list[i])
plt.show()

In [ ]:
true_silh = silhouette_score(U_train, c_train)
pcmf_silh = silhouette_score(pcmf_U, c_train)
spcmf_silh = silhouette_score(spcmf_U, c_train)
hpcmf_silh = silhouette_score(hpcmf_U, c_train)
shpcmf_silh = silhouette_score(shpcmf_U, c_train)
pca_silh = silhouette_score(pca_U, c_train)

scores = {'SPCMF': spcmf_silh, 'PCMF': pcmf_silh, 'PCA': pca_silh, 'HPCMF': hpcmf_silh, 'SHPCMF': shpcmf_silh}

sorted_scores = sorted(scores.items(), key=operator.itemgetter(1), reverse=True)

print('Silhouette scores (higher is better):')
print('\033[1m- {0}: {1:.3}\033[0m'.format(sorted_scores[0][0], sorted_scores[0][1]))
for score_tp in sorted_scores[1:]:
    print('- {0}: {1:.3}'.format(score_tp[0], score_tp[1]))
    
print('\nSilhouette of true U:')
print('%0.3f' % true_silh)